In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2, ResNet50
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.models import Model

In [11]:
path = glob('/content/drive/MyDrive/images/images/*.xml')

labels_dict = dict(filepath=[], xmin=[], xmax=[], ymin=[], ymax=[])

for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [12]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/images/images/N101.xml,31,139,128,161
1,/content/drive/MyDrive/images/images/N102.xml,164,316,216,243
2,/content/drive/MyDrive/images/images/N106.xml,137,262,249,290
3,/content/drive/MyDrive/images/images/N108.xml,184,342,220,257
4,/content/drive/MyDrive/images/images/N100.xml,134,301,312,350


In [13]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/content/drive/MyDrive/images12/images', filename_image)
    return filepath_image
getFilename(filename)

'/content/drive/MyDrive/images12/images/N101.jpeg'

In [ ]:
image_path = list(df['filepath'].apply(getFilename))


In [ ]:
file_path = image_path[87]
img = cv2.imread(file_path)

img = io.imread(file_path)
fig = px.imshow(img)
fig.update_layout(width=500, height=268, margin=dict(l=10, r=10, b=10, t=10))
fig.add_shape(type='rect',x0=392, x1=517, y0=161, y1=213, xref='x', yref='y',line_color='cyan')

In [ ]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape

    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0

    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax)

    data.append(norm_load_image_arr)
    output.append(label_norm)

In [ ]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)

x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

In [ ]:
resnet = ResNet50(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))

headmodel = resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)



model = Model(inputs=resnet.input,outputs=headmodel)

In [ ]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4))
model.summary()

In [ ]:
history = model.fit(x=x_train, y=y_train, batch_size=8, epochs=100,
                    validation_data=(x_test,y_test))

Epoch 1/100
23/23 [==============================] - 5s 201ms/step - loss: 1.7429e-04 - val_loss: 0.0110
Epoch 2/100
23/23 [==============================] - 5s 206ms/step - loss: 2.0393e-04 - val_loss: 0.0091
Epoch 3/100
23/23 [==============================] - 5s 214ms/step - loss: 2.9472e-04 - val_loss: 0.0094
Epoch 4/100
23/23 [==============================] - 5s 229ms/step - loss: 1.9361e-04 - val_loss: 0.0096
Epoch 5/100
23/23 [==============================] - 5s 203ms/step - loss: 1.6589e-04 - val_loss: 0.0085
Epoch 6/100
23/23 [==============================] - 5s 217ms/step - loss: 1.5546e-04 - val_loss: 0.0091
Epoch 7/100
23/23 [==============================] - 5s 224ms/step - loss: 1.4543e-04 - val_loss: 0.0079
Epoch 8/100
23/23 [==============================] - 5s 205ms/step - loss: 1.8870e-04 - val_loss: 0.0078
Epoch 9/100
23/23 [==============================] - 5s 222ms/step - loss: 1.6818e-04 - val_loss: 0.0085
Epoch 10/100
23/23 [==============================] - 5

In [ ]:
path = '/content/drive/MyDrive/images12/TEST/TEST.jpeg'
def object_detection(path):


    image = load_img(path)
    image = np.array(image,dtype=np.uint8)
    image1 = load_img(path,target_size=(224,224))


    image_arr_224 = img_to_array(image1)/255.0
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)


    coords = model.predict(test_arr)


    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)


    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

image, cods = object_detection(path)

fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10))

1/1 [==============================] - 0s 37ms/step
(318, 488) (600, 556)
